In [252]:
import numpy as np
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy

In [253]:
nlp = spacy.load('en_core_web_sm')

In [254]:
train = pd.read_csv('C:\\Users\\Faycal\\Downloads\\nlp-getting-started\\train.csv')
test = pd.read_csv('C:\\Users\\Faycal\\Downloads\\nlp-getting-started\\test.csv')

In [292]:
print(train.iloc[0,3])


Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all


In [293]:
doc = nlp(train.iloc[0,3])
displacy.render(doc,style = "ent")

In [294]:
doc2 = nlp(train.iloc[1,3])
displacy.render(doc2,style = "ent")

In [255]:
with nlp.disable_pipes():
    train_vectors = np.array([nlp(text).vector for text in train.text])
    test_vectors = np.array([nlp(text).vector for text in test.text])

print(train_vectors.shape, test_vectors.shape)

(7613, 96) (3263, 96)


In [256]:
from sklearn.model_selection import train_test_split
X_train = train_vectors
y_train = train.target.to_numpy()

train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.1)

In [257]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,
GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

svc = SVC(kernel='rbf', C=70, gamma='auto', probability=True, random_state=42)
rfc = RandomForestClassifier(n_estimators =100,random_state=42)
xgbc =  XGBClassifier(n_estimators=950, learning_rate=0.2, random_state=42)

In [258]:
estimators = [svc, rfc,xgbc]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=False)
Training the RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n

In [259]:
from sklearn.ensemble import VotingClassifier
vcf = VotingClassifier(estimators=[('svc', svc), ('rfc', rfc), ('xgbc', xgbc)], voting='soft')
vcf.fit(X_train, y_train)

VotingClassifier(estimators=[('svc',
                              SVC(C=70, cache_size=200, class_weight=None,
                                  coef0=0.0, decision_function_shape='ovr',
                                  degree=3, gamma='auto', kernel='rbf',
                                  max_iter=-1, probability=True,
                                  random_state=42, shrinking=True, tol=0.001,
                                  verbose=False)),
                             ('rfc',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='au...
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                  

In [260]:
from sklearn.metrics import accuracy_score
preds = vcf.predict(test_vectors)
print(accuracy_score((vcf.predict(test_x)), test_y))
print(len(test['id']), len(preds))

0.9993434011818779
3263 3263


In [261]:
[estimator.score(test_x, test_y) for estimator  in estimators]

[0.9868680236375574, 0.9993434011818779, 0.9993434011818779]

In [262]:
preds = rfc.predict(test_vectors)
print(accuracy_score((rfc.predict(test_x)), test_y))
print(len(test['id']), len(preds))

0.9993434011818779
3263 3263


In [263]:
submission = pd.DataFrame(columns=['id', 'target'])
submission['id'] = test['id']
submission['target'] = preds
submission.to_csv('submission.csv', index=False)

In [278]:
first_tweet = train.iloc[0,3]
print(first_tweet)

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
